In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objs as go
from plotly.subplots import make_subplots

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

## Создаем датасет

In [2]:
n_samples = 20000
x = np.random.uniform(-10, 10, n_samples)
y = np.random.uniform(-10, 10, n_samples)
z = np.sin(x + 2*y) * np.exp(-(2*x + y)**2)

In [3]:
X = np.vstack((x, y)).T
X

array([[-2.50919762,  4.59996622],
       [ 9.01428613, -6.30976009],
       [ 4.63987884, -3.06720611],
       ...,
       [-3.92603062, -4.60861136],
       [-1.13359987, -1.31360456],
       [-6.55470371, -0.25152603]])

## Делим на выборки

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, z, test_size=0.3, random_state=RANDOM_SEED)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=RANDOM_SEED)

In [5]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [6]:
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.l1 = nn.Linear(2, 128)
        self.l2 = nn.Linear(128, 128)
        self.l3 = nn.Linear(128, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.l1(x))
        x = self.relu(self.l2(x))
        x = self.l3(x)
        return x

## Обучение

In [7]:
model = RegressionModel()
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
num_epochs = 100

for epoch in range(num_epochs):
    model.train()

    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = loss_fn(val_outputs, y_val)

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

Epoch [10/100], Loss: 0.0379, Val Loss: 0.0294
Epoch [20/100], Loss: 0.0212, Val Loss: 0.0218
Epoch [30/100], Loss: 0.0210, Val Loss: 0.0198
Epoch [40/100], Loss: 0.0190, Val Loss: 0.0171
Epoch [50/100], Loss: 0.0179, Val Loss: 0.0164
Epoch [60/100], Loss: 0.0178, Val Loss: 0.0164
Epoch [70/100], Loss: 0.0176, Val Loss: 0.0160
Epoch [80/100], Loss: 0.0175, Val Loss: 0.0159
Epoch [90/100], Loss: 0.0174, Val Loss: 0.0159
Epoch [100/100], Loss: 0.0172, Val Loss: 0.0158


## Считаем MSE на test

In [9]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = loss_fn(test_outputs, y_test)
    print(f'Test MSE: {test_loss.item():.4f}')

Test MSE: 0.0172


## График истинной и предсказанной функции

In [10]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]],
                    subplot_titles=['True Function', 'Predicted Function'])

# Добавление точек данных на график истинной функции
true_scatter = go.Scatter3d(x=X_test[:, 0], y=X_test[:, 1], z=y_test[:, 0], mode='markers', marker=dict(size=2, color='black'))
fig.add_trace(true_scatter, row=1, col=1)

# Добавление точек данных на график предсказанной функции
pred_scatter = go.Scatter3d(x=X_test[:, 0], y=X_test[:, 1], z=test_outputs[:, 0], mode='markers', marker=dict(size=2, color='red'))
fig.add_trace(pred_scatter, row=1, col=2)

# Настройки визуализации
fig.update_layout(title='True Function vs Predicted Function',
                  scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
                  scene2=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'))
fig.show()


![Alt text](result_plot.png)